Requirements
For Project 3, you will work with your group to tell a story using data visualizations. Here are the specific requirements:

Your visualization must include a Python Flask-powered API, HTML/CSS, JavaScript, and at least one database (SQL, MongoDB, SQLite, etc.).

Your project should fall into one of the following three tracks:

A combination of web scraping and Leaflet or Plotly

A dashboard page with multiple charts that update from the same data

A server that performs multiple manipulations on data in a database prior to visualization (must be approved)

Your project should include at least one JS library that we did not cover.

Your project must be powered by a dataset with at least 100 records.

Your project must include some level of user-driven interaction (e.g., menus, dropdowns, textboxes).

Your final visualization should ideally include at least three views.

In [4]:
#all dependecias imports
import pprint
from amadeus import Client, ResponseError
import logging
from amadeus_keys import client_id, client_secret
from flask import Flask, jsonify, render_template
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func



In [5]:
#make an API call and visualize it using Flask API
app = Flask(__name__) # need to create a file for the app to set up the application 

amadeus = Client(
    client_id=client_id,
    client_secret=client_secret
)

locations = [
    {'latitude': 40.7128, 'longitude': -74.0060, 'name': 'New York City'},
    {'latitude': 40.9136005, 'longitude': -73.8290816, 'name': 'Mount Vernon'},
    {'latitude': 40.033985, 'longitude': -73.762909, 'name': 'White Plains'},
    {'latitude': 40.9025, 'longitude': -73.9247, 'name': 'Yonkers'},
    {'latitude': 40.9232, 'longitude': -73.7793, 'name': 'New Rochelle'},
    {'latitude': 40.7127, 'longitude': -73.0737, 'name': 'Jersey City'},
    {'latitude': 40.7094091, 'longitude': -74.2095504, 'name': 'Newark'},
    {'latitude': 41.1936893070528, 'longitude': -73.1972645346039, 'name': 'Bridgeport'},
    {'latitude': 41.0376034, 'longitude': -73.6243386, 'name': 'Greenwich'}
]

# Define your SQLAlchemy and Amadeus configuration and setup here

@app.route('/')
def index():
    try:
        data = []
        for location in locations:
            response = amadeus.reference_data.locations.points_of_interest.get(
                latitude=location['latitude'],
                longitude=location['longitude']
            )
            points_of_interest = response.data

            location_data = {
                'name': location['name'],
                'points_of_interest': points_of_interest
            }
            data.append(location_data)

        return jsonify(data)

    except ResponseError as error:
        return jsonify({'error': str(error)})

if __name__ == '__main__':
    app.run(debug=true)

NameError: name 'true' is not defined

In [ ]:
#make an API call and visualize it using Flask API
app = Flask(__name__)

# reflect the database tables
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

# Save references to each table
Restaurant = Base.classes.restaurant
Sights = Base.classes.sights
Shopping = Base.classes.shopping

# Create our session (link) from Python to the DB
session = Session(engine)




amadeus = Client(
    client_id=client_id,
    client_secret=client_secret
)

locations = [
    {'latitude': 40.7128, 'longitude': -74.0060, 'name': 'New York City'},
    {'latitude': 40.9136005, 'longitude': -73.8290816, 'name': 'Mount Vernon'},
    {'latitude': 40.033985, 'longitude': -73.762909, 'name': 'White Plains'},
    {'latitude': 40.9025, 'longitude': -73.9247, 'name': 'Yonkers'},
    {'latitude': 40.9232, 'longitude': -73.7793, 'name': 'New Rochelle'},
    {'latitude': 40.7127, 'longitude': -73.0737, 'name': 'Jersey City'},
    {'latitude': 40.7094091, 'longitude': -74.2095504, 'name': 'Newark'},
    {'latitude': 41.1936893070528, 'longitude': -73.1972645346039, 'name': 'Bridgeport'},
    {'latitude': 41.0376034, 'longitude': -73.6243386, 'name': 'Greenwich'}
]

try:
    for location in locations:
        response = amadeus.reference_data.locations.points_of_interest.get(
            latitude=location['latitude'],
            longitude=location['longitude']
        )
        points_of_interest = response.data
        print(f"Points of interest for {location['name']}:")
        for poi in points_of_interest:
            print("Name:", poi['name'])
            print("Category:", poi['category'])
            print("Tags:", poi['tags'])
            print()  # Separate each point of interest by a blank line
        print()

except ResponseError as error:
    print(error.response.body)
    raise error




In [7]:
words = ['mall', 'shopping', 'sightseeing', 'museum', 'sights', 'building', 'commercialplace', 'landmark', 'tourguide', 'clothing', 'professionalservices', 'activities']

total_attractions = 0
for word in words:
    total_attractions += len(word)

print(total_attractions)



113
